In [1]:
!pip install xgboost

Looking in indexes: https://pypi.mirrors.ustc.edu.cn/simple/
  Retrying (Retry(total=4, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='mirrors.ustc.edu.cn', port=443): Read timed out. (read timeout=15)",)': /pypi/web/simple/xgboost/
    100% |████████████████████████████████| 142.8MB 252kB/s eta 0:00:01


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold
import datetime
import warnings
import pickle as pkl
import xgboost as xgb

warnings.filterwarnings('ignore')

In [3]:
with open('/home/aistudio/data/data13514/feature_embed.p','rb') as source:
    df = pkl.load(source)

print('加载完成', df.shape)

加载完成 (374953, 110)


In [4]:
df = pd.DataFrame(df)

In [5]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,100,101,102,103,104,105,106,107,108,109
0,0.8,2,2,680,680,0,0,510,0,0,...,0,0,1,0,1,0,0,0,1,0
1,0.4,2,2,500,500,0,0,330,0,0,...,0,0,1,1,0,0,0,1,0,0
2,0.4,1,1,420,420,0,0,420,0,0,...,0,0,1,1,0,0,0,1,0,0
3,0.4,3,3,785,785,0,0,85,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0.4,5,5,1529.98,1529.98,0,0,610,0,0,...,0,0,0,1,0,0,0,1,0,1


In [6]:
def logloss(y_true, y_pred):
    deta = 3
    eps=1e-15
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred[:,1])
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss/len(y_true)
def singleloss(y_true, y_pred):
    deta = 3
    eps=1e-15
    # Prepare numpy array data
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return loss/len(y_true)
def evalerror(preds, dtrain):
    deta = 3
    eps=1e-15
    labels = dtrain.get_label()
    y_true = np.array(labels)
    y_pred = np.array(preds)
    assert (len(y_true) and len(y_true) == len(y_pred))
    # Clip y_pred between eps and 1-eps
    p = np.clip(y_pred, eps, 1-eps)
    loss = np.sum(- y_true * np.log(p) * deta - (1 - y_true) * np.log(1-p))
    return 'error', loss/len(y_true)

In [7]:
logScore = make_scorer(logloss, greater_is_better=True, needs_proba=True)
scoring = {'logloss':logScore}

In [8]:
print('划分训练测试集')
feature = df.iloc[:,:-1].values
label = df.iloc[:,-1].values
label.reshape(-1,1)
print(feature.shape)
print(label.shape)
x_train, x_valid, y_train, y_valid = train_test_split(feature, label, test_size=0.2, stratify=label)
train = xgb.DMatrix(data=feature, label=label)
valid = xgb.DMatrix(data=x_valid, label=y_valid)

划分训练测试集
(374953, 109)
(374953,)


In [65]:
params = {
    'booster': 'gbtree',
    'objective':'binary:logistic',
    'max_depth':12,
    'eta':0.01,
    'silent':1,
    'gamma':0.2,
    'min_child_weight':8,
    'subsample':1,
    'colsample_bytree':1,
    'colsample_bylevel':1,
    'lambda':0.6,
    'alpha':0,
    'nthread':-1,
    # 'eval_metric':logScore,
    'seed':2018
}
test = [(valid,'eval')]

In [66]:
model = xgb.train(params=params, dtrain=train, num_boost_round=20000, feval=evalerror, evals=test, early_stopping_rounds=10)

[0]	eval-error:0.149458	eval-error:0.914799
Multiple eval metrics have been passed: 'eval-error' will be used for early stopping.

Will train until eval-error hasn't improved in 10 rounds.
[1]	eval-error:0.148965	eval-error:0.91091
[2]	eval-error:0.148778	eval-error:0.907121
[3]	eval-error:0.148738	eval-error:0.903436
[4]	eval-error:0.148338	eval-error:0.899839
[5]	eval-error:0.148271	eval-error:0.896327
[6]	eval-error:0.148218	eval-error:0.892912
[7]	eval-error:0.147644	eval-error:0.889584
[8]	eval-error:0.147644	eval-error:0.886339
[9]	eval-error:0.147551	eval-error:0.883169
[10]	eval-error:0.147471	eval-error:0.880084
[11]	eval-error:0.147298	eval-error:0.877069
[12]	eval-error:0.147031	eval-error:0.874132
[13]	eval-error:0.146991	eval-error:0.871274
[14]	eval-error:0.146764	eval-error:0.868483
[15]	eval-error:0.146818	eval-error:0.865764
[16]	eval-error:0.146818	eval-error:0.863124
[17]	eval-error:0.146604	eval-error:0.860541
[18]	eval-error:0.146458	eval-error:0.858016
[19]	eval-e

In [69]:
model.save_model('work/my_model.model')
model = xgb.Booster()
model.load_model('work/my_model.model')
print('提交前预测结果（0.771093）')
res = model.predict(xgb.DMatrix(x_valid))
print(res)
print(singleloss(y_valid,res))

提交前预测结果（0.771093）
[0.18703382 0.18690039 0.26751837 ... 0.35541648 0.21331905 0.19868453]
0.7772550531941499


In [68]:
cv = xgb.cv(params, train, num_boost_round=1000, nfold=5, feval=evalerror, early_stopping_rounds=10, seed=0)
print('Best round：',cv.shape[0])
print('Best score：',cv.values[-1,:])

KeyboardInterrupt: 

In [27]:
model = xgb.XGBClassifier(max_depth=12, learning_rate=0.1, n_estimators=19, silent=True, 
                         objective='binary:logistic', nthread=-1, gamma=0.2, min_child_weight=8, 
                         max_delta_step=0, subsample=0.9, colsample_bytree=0.9, colsample_bylevel=0.9, 
                         reg_alpha=0., reg_lambda=0.6, scale_pos_weight=2.5, base_score=0.5, seed=0, 
                         missing=None)
model.fit(feature,label)
res = model.predict_proba(x_valid)
print(res)
print(logloss(y_valid,res))

[[0.80034995 0.19965006]
 [0.8144449  0.18555509]
 [0.6479087  0.3520913 ]
 ...
 [0.5023437  0.4976563 ]
 [0.70095223 0.29904777]
 [0.778038   0.221962  ]]
0.7150601975202411


In [25]:
params = {
    # 'learning_rate':[0.1,0.05,0.03,0.01],
    # 'n_estimatores':np.arange(100,400,100),
    # 'min_child_weight':np.arange(7,14,1),
    # 'max_depth':np.arange(8,15,1),
    # 'gamma':np.arange(0,0.5,.1),
    # 'subsample':[0.5,0.6,0.7,0.8,0.9,1],# np.arange(0.5,1.1,.1),
    # 'colsample_bytree':[0.5,0.6,0.7,0.8,0.9,1],# np.arange(0.5,1.1,.1),
    # 'colsample_bylevel':[0.5,0.6,0.7,0.8,0.9,1],# np.arange(0.5,1.1,.1),
    # 'reg_alpha':[0.,0.1,0.2], # np.arange(0,0.5,.1),
    # 'reg_lambda':[0.5,0.6,0.7,0.8,0.9,1], # np.arange(0.5,1.1,.1),
}

cv = GridSearchCV(model, param_grid=params, scoring=logScore, n_jobs=-1, cv=3, verbose=1)

cv.fit(feature, label)
print('Best score:',cv.best_score_)
print('Best param:',cv.best_params_)

Fitting 3 folds for each of 7 candidates, totalling 21 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  21 out of  21 | elapsed:  3.1min finished


Best score: 0.8497743090809315
Best param: {'min_child_weight': 8}


In [70]:
with open('/home/aistudio/data/data13514/eature_embed_all.','rb') as source:
    features = pkl.load(source)

In [71]:
features.shape

(929807, 109)

In [72]:
features = pd.DataFrame(features)

In [73]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,0.8,2,2,680,680,0,0,510,0,0,...,0,0,0,1,0,1,0,0,0,1
1,0.4,2,2,500,500,0,0,330,0,0,...,0,0,0,1,1,0,0,0,1,0
2,0.4,1,1,420,420,0,0,420,0,0,...,0,0,0,1,1,0,0,0,1,0
3,0.4,3,3,785,785,0,0,85,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0.4,63,68,20042.9,19299.8,0.0149549,0.0155307,300,0,0,...,1,0,0,0,1,0,0,0,1,0


In [74]:
features['predict'] = 0

In [75]:
features['predict'] = model.predict(xgb.DMatrix(features.iloc[:,:-1].values))
# features['predict'] = model.predict_proba(features.iloc[:,:-1].values)[:,1]

In [76]:
output = features[[64,'predict']]
output = output.rename({64:'customer_id','predict':'result'},axis=1)
output.to_csv('reuslt.csv',index=False)

In [77]:
output.shape

(929807, 2)

In [78]:
output

,customer_id,result
0,133,0.322463
1,413,0.234745
2,113,0.214413
3,502,0.291103
4,0,0.698398
5,7,0.623874
6,8,0.623874
7,17,0.337641
8,55,0.191385
9,73,0.222739
